In [5]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [7]:
# load ascii text and covert to lowercase
filename = "FleursDuMal.txt"
raw_text = open(filename, encoding="utf8").read()
raw_text = raw_text.lower()

Must convert characters to integer

In [8]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [9]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144941
Total Vocab:  56


In [10]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144841


In [11]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [12]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

W0725 11:32:35.676050  3436 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0725 11:32:35.689016  3436 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0725 11:32:35.691012  3436 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0725 11:32:36.003172  3436 deprecation_wrapper.py:119] From E:\Anaconda\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0725 11:32:36.010154  3436 deprecation.py:506] From E:\Anaconda\lib\site-packages\keras\backend\t

In [13]:
# define the checkpoint
filepath="weights-FdM-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [14]:
#model.load_weights('weights-conan-338-0.3525.hdf5')
model.fit(X, y, epochs=10000, batch_size=64, callbacks=callbacks_list)

W0725 11:32:39.157090  3436 deprecation.py:323] From E:\Anaconda\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10000
144841/144841 [==============================] - 11795s 81ms/step - loss: 2.6985

Epoch 00001: loss improved from inf to 2.69847, saving model to weights-FdM-01-2.6985.hdf5
Epoch 2/10000
144841/144841 [==============================] - 12154s 84ms/step - loss: 2.2740

Epoch 00002: loss improved from 2.69847 to 2.27397, saving model to weights-FdM-02-2.2740.hdf5
Epoch 3/10000
144841/144841 [==============================] - 12013s 83ms/step - loss: 2.0460

Epoch 00003: loss improved from 2.27397 to 2.04600, saving model to weights-FdM-03-2.0460.hdf5
Epoch 4/10000
144841/144841 [==============================] - 12006s 83ms/step - loss: 1.8801

Epoch 00004: loss improved from 2.04600 to 1.88005, saving model to weights-FdM-04-1.8801.hdf5
Epoch 5/10000
144841/144841 [==============================] - 11971s 83ms/step - loss: 1.7437

Epoch 00005: loss improved from 1.88005 to 1.74369, saving model to weights-FdM-05-1.7437.hdf5
Epoch 6/10000
144841/144841 [====================

KeyboardInterrupt: 

In [9]:
# load the network weights
filename = "weights-Fdm-165-0.6485.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [25]:
import sys

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
#print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
ed ligenn and mands oa ladrest were ruehtere whsh derteon samagl reacrtert th t toraha mened anhin the pratss in stiels  and the
lltedom of the plder saae hi the hyrkanians, nuweaned byoger oe ã†rir and vanir swarmed into the lands, and the pictish eslans and thnugm poprter, the brrnindaney  femecin whrh the besserians ale sordicas whi hape intl teemr line tmet ay the hererialis of sueges cadr on the rears of tae sraless aadrea toygia  nnten thay horg the lars thav ate tham th ted wivh thets la wac sestey oftoon weto crue ly andis, the
newer kingdom is called ntygiaa ir anuilonia and wast elcatey, the
western world. rhiue werl
nohrinle iyteriory ges ont came tn eupse th  soun of the basbarians of
the aasbarians hf
tch saaee- tiey iad alrn eo name tal and plunder. the racr of the hypkanian kiogs.ms. 
shil and tavages whah asunlunia'aad wete botthered enmmle iioo mtrerilg banore them, theu siey nor ralas abee to mheeu, which had eer tortin sort wort clpeled trth the nome cabkai aro